https://dedupeio.github.io/dedupe-examples/docs/csv_example.html

This code demonstrates how to use dedupe with a comma separated values (CSV) file. All operations are performed in memory, so will run very quickly on datasets up to ~10,000 rows.

We start with a CSV file containing our messy data. In this example, it is listings of early childhood education centers in Chicago compiled from several different sources.

The output will be a CSV with our clustered results.

In [1]:
import os
import csv
import re
import logging
import optparse
from future.builtins import next
import dedupe
from unidecode import unidecode

# Logging
Dedupe uses Python logging to show or suppress verbose output. This code block lets you change the level of loggin on the command line. You don't need it if you don't want that. To enable verbose logging, run python examples/csv_example/csv_example.py -v

In [2]:
optp = optparse.OptionParser()
optp.add_option('-v', '--verbose', dest='verbose', action='count',
                help='Increase verbosity (specify multiple times for more)'
                )
(opts, args) = optp.parse_args()
log_level = logging.WARNING 
if opts.verbose:
    if opts.verbose == 1:
        log_level = logging.INFO
    elif opts.verbose >= 2:
        log_level = logging.DEBUG
logging.getLogger().setLevel(log_level)

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Setup
Do a little bit of data cleaning with the help of Unidecode and Regex. Things like casing, extra spaces, quotes and new lines can be ignored.  If data is missing, indicate that by setting the value to None

In [3]:
os.getcwd()

'/home/jovyan/work'

In [46]:
csv_file_directory = '/home/jovyan/work/csv_files/'
input_file = csv_file_directory + 'csv_customer_input.csv'
#input_file = csv_file_directory + 'csv_example_messy_input.csv' 
output_file = csv_file_directory+ 'csv_customer_output.csv'

In [47]:
settings_directory = '/home/jovyan/work/training_settings/'
settings_file = settings_directory + 'csv_example_learned_settings'
training_file = settings_directory + 'csv_example_training.json'

In [48]:
# directory = '/home/jovyan/work/dedupe/dedupe-examples/csv_example/'
# input_file = directory + 'csv_example_messy_input.csv'
# output_file = directory + 'csv_example_output.csv'
# settings_file = directory + 'csv_example_learned_settings'
# training_file = directory + 'csv_example_training.json'

In [49]:
def preProcess(column):
    try : # python 2/3 string differences
        column = column.decode('utf8')
    except AttributeError:
        pass
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    
    if not column:
        column = None
    return column

In [66]:
import sys

def readData(filename):
#
    data_d = {}
    with open(filename, encoding="utf-8-sig") as f:
        reader = csv.DictReader(f)
        rowcount = 0
        try:
            for row in reader:
                rowcount += 1
                clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
                row_id = int(row['Id'])
                data_d[row_id] = dict(clean_row)
        except:
            print("Unexpected error:", sys.exc_info()[0])
            print("Row %d dump:" % (rowcount))
            print(row)

    return data_d

In [67]:
print('importing data ...')
data_d = readData(input_file)

importing data ...


In [68]:
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    
    
    
    fields = [
        {'field' : 'Account_Name', 'type': 'String'},
        {'field' : 'Billing', 'type': 'String'},
        {'field' : 'Industry', 'type': 'String'},
        ]
    deduper = dedupe.Dedupe(fields)
    
    
    deduper.sample(data_d, 15000)
    
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.readTraining(f)
            
    print('starting active labeling...')

INFO:dedupe.canopy_index:Removing stop word  t
INFO:dedupe.canopy_index:Removing stop word es
INFO:dedupe.canopy_index:Removing stop word nc
INFO:dedupe.canopy_index:Removing stop word ri
INFO:dedupe.canopy_index:Removing stop word s 
INFO:dedupe.canopy_index:Removing stop word to
INFO:dedupe.canopy_index:Removing stop word an
INFO:dedupe.canopy_index:Removing stop word el
INFO:dedupe.canopy_index:Removing stop word sa
INFO:dedupe.canopy_index:Removing stop word  a
INFO:dedupe.canopy_index:Removing stop word er
INFO:dedupe.canopy_index:Removing stop word li
INFO:dedupe.canopy_index:Removing stop word ne
INFO:dedupe.canopy_index:Removing stop word r 
INFO:dedupe.canopy_index:Removing stop word un
INFO:dedupe.canopy_index:Removing stop word o 
INFO:dedupe.canopy_index:Removing stop word ro
INFO:dedupe.canopy_index:Removing stop word le
INFO:dedupe.canopy_index:Removing stop word tr
INFO:dedupe.canopy_index:Removing stop word  c
INFO:dedupe.canopy_index:Removing stop word al
INFO:dedupe.c

starting active labeling...


In [69]:
dedupe.consoleLabel(deduper)

Account_Name : adetel groupe
Billing : None
Industry : banking & insurance

Account_Name : agena gmbh
Billing : None
Industry : technology

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


Account_Name : boustead naval shipyard sdn bhd
Billing : None
Industry : transportation

Account_Name : efficient softech sdn bhd
Billing : asean
Industry : technology

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


Account_Name : bridgestone
Billing : None
Industry : None

Account_Name : bridgestone
Billing : sao paulo
Industry : manufacturing

0/10 positive, 2/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (commonSixGram, Account_Name), SimplePredicate: (wholeFieldPredicate, Account_Name))
INFO:dedupe.training:(SimplePredicate: (hundredIntegersOddPredicate, Account_Name), SimplePredicate: (sameSevenCharStartPredicate, Account_Name))
INFO:dedupe.training:(TfidfNGramCanopyPredicate: (0.2, Billing), TfidfTextCanopyPredicate: (0.6, Account_Name))
INFO:dedupe.training:(SimplePredicate: (twoGramFingerprint, Industry), SimplePredicate: (wholeFieldPredicate, Account_Name))
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, Account_Name), SimplePredicate: (doubleMetaphone, Account_Name))
Account_Name : adama
Billing : na
Industry : chemicals

Account_Name : adama india private limited
Billing : telangana
Industry : chemicals

11/10 positive, 17/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


Account_Name : randstad pty limited (vedior asia pacific pty. limited)
Billing : nsw
Industry : consulting

Account_Name : randstad usa
Billing : ma
Industry : consulting

12/10 positive, 17/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, Account_Name), TfidfTextCanopyPredicate: (0.4, Account_Name))
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, Account_Name), SimplePredicate: (doubleMetaphone, Account_Name))
INFO:dedupe.training:(SimplePredicate: (hundredIntegersOddPredicate, Account_Name), SimplePredicate: (sameSevenCharStartPredicate, Account_Name))
INFO:dedupe.training:(TfidfNGramCanopyPredicate: (0.2, Billing), TfidfTextCanopyPredicate: (0.6, Account_Name))


In [70]:
deduper.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.5570857334420573
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (sameFiveCharStartPredicate, Account_Name), TfidfTextCanopyPredicate: (0.4, Account_Name))
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, Account_Name), SimplePredicate: (doubleMetaphone, Account_Name))
INFO:dedupe.training:(SimplePredicate: (hundredIntegersOddPredicate, Account_Name), SimplePredicate: (sameSevenCharStartPredicate, Account_Name))
INFO:dedupe.training:(TfidfNGramCanopyPredicate: (0.2, Billing), TfidfTextCanopyPredicate: (0.6, Account_Name))


In [71]:
with open(training_file, 'w') as tf:
        deduper.writeTraining(tf)

In [72]:
 with open(settings_file, 'wb') as sf:
        deduper.writeSettings(sf)

In [73]:
threshold = deduper.threshold(data_d, recall_weight=1)

INFO:dedupe.canopy_index:Removing stop word of
INFO:dedupe.canopy_index:Removing stop word inc
INFO:dedupe.canopy_index:Removing stop word ltd
INFO:dedupe.blocking:10000, 1.3983642 seconds
INFO:dedupe.api:Maximum expected recall and precision
INFO:dedupe.api:recall: 0.851
INFO:dedupe.api:precision: 0.805
INFO:dedupe.api:With threshold: 0.414


In [74]:
print('clustering...')
clustered_dupes = deduper.match(data_d, threshold)

print('# duplicate sets', len(clustered_dupes))

clustering...


INFO:dedupe.canopy_index:Removing stop word of
INFO:dedupe.canopy_index:Removing stop word inc
INFO:dedupe.canopy_index:Removing stop word ltd
INFO:dedupe.blocking:10000, 1.7214772 seconds


# duplicate sets 843


In [75]:
cluster_membership = {}
cluster_id = 0
for (cluster_id, cluster) in enumerate(clustered_dupes):
    id_set, scores = cluster
    cluster_d = [data_d[c] for c in id_set]
    canonical_rep = dedupe.canonicalize(cluster_d)
    for record_id, score in zip(id_set, scores):
        cluster_membership[record_id] = {
            "cluster id" : cluster_id,
            "canonical representation" : canonical_rep,
            "confidence": score
        }

In [76]:
singleton_id = cluster_id + 1

In [77]:
with open(output_file, 'w') as f_output, open(input_file) as f_input:
    writer = csv.writer(f_output)
    reader = csv.reader(f_input)

    heading_row = next(reader)
    heading_row.insert(0, 'confidence_score')
    heading_row.insert(0, 'Cluster ID')
    canonical_keys = canonical_rep.keys()
    for key in canonical_keys:
        heading_row.append('canonical_' + key)

    writer.writerow(heading_row)

    for row in reader:
        row_id = int(row[0])
        if row_id in cluster_membership:
            cluster_id = cluster_membership[row_id]["cluster id"]
            canonical_rep = cluster_membership[row_id]["canonical representation"]
            row.insert(0, cluster_membership[row_id]['confidence'])
            row.insert(0, cluster_id)
            for key in canonical_keys:
                row.append(canonical_rep[key].encode('utf8'))
        else:
            row.insert(0, None)
            row.insert(0, singleton_id)
            singleton_id += 1
            for key in canonical_keys:
                row.append(None)
        writer.writerow(row)

In [78]:
import pandas as pd

In [79]:
dfOutput = pd.read_csv(output_file)
dfOutput.head(20)

,Cluster ID,confidence_score,﻿Id,id_2,Source,Account_Name,Billing,Industry,canonical_Id,canonical_id_2,canonical_Source,canonical_Account_Name,canonical_Billing,canonical_Industry
0,843,NaN,0,1_LW,_LW,"1-800-Battery, Inc",California,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,844,NaN,1,2 LW,LW,1SYNC,New Jersey,Technology,NaN,NaN,NaN,NaN,NaN,NaN
2,845,NaN,2,3 LW,LW,3M (fka CodeRyte),Maryland,Technology,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0.991215,3,4 LW,LW,A1 Telekom Austria AG,Vienna,Telecommunications,b'3',b'4 lw',b'lw',b'a1 telekom austria ag',b'vienna',b'telecommunications'
4,846,NaN,4,5 LW,LW,AbbVie Inc.,Illinois,Healthcare & Pharmaceuticals,NaN,NaN,NaN,NaN,NaN,NaN
5,847,NaN,5,6 LW,LW,Accenture Inc.,Illinois,"Services,Technology",NaN,NaN,NaN,NaN,NaN,NaN
6,848,NaN,6,7 LW,LW,ACE Group,Pennsylvania,Financial,NaN,NaN,NaN,NaN,NaN,NaN
7,2,0.946273,7,8 LW,LW,Ace Hardware Corporation,Illinois,Services,b'7',b'8 lw',b'lw',b'ace hardware corporation',b'illinois',b'services'
8,849,NaN,8,9 LW,LW,Aerospace Corporation,California,Consulting,NaN,NaN,NaN,NaN,NaN,NaN
9,850,NaN,9,10 LW,LW,Aetna Inc.,Connecticut,"Financial,Healthcare & Pharmaceuticals",NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
#dfOutput.groupby(['Cluster ID']).count()
dfOutput.to_csv(